# volatility model using ARCH , GARCH and EWMA #

In [2]:
# import imp lib 

import yfinance as yf 
import numpy as np 
import pandas as pd 



In [3]:
# import data from yfinace 

df=yf.download('JPM',start='2022-01-01', end='2025-01-01')
df=df['Close']
df

C:\Users\DELL\AppData\Local\Temp\ipykernel_8028\3900761540.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df=yf.download('JPM',start='2022-01-01', end='2025-01-01')
[*********************100%***********************]  1 of 1 completed


Ticker,JPM
Date,
2022-01-03,146.291046
2022-01-04,151.836884
2022-01-05,149.060989
2022-01-06,150.644638
2022-01-07,152.137238
...,...
2024-12-24,238.440521
2024-12-26,239.257263
2024-12-27,237.318710


# ARCH

In [4]:
from arch import arch_model

In [5]:
# calculate daily return

df['returns']=df.pct_change()*100

returns= df['returns'].dropna()
returns

Date
2022-01-04    3.790962
2022-01-05   -1.828208
2022-01-06    1.062417
2022-01-07    0.990808
2022-01-10    0.095723
                ...   
2024-12-24    1.644367
2024-12-26    0.342535
2024-12-27   -0.810238
2024-12-30   -0.767091
2024-12-31    0.162962
Name: returns, Length: 752, dtype: float64

In [6]:
# create and fit ARCH model

model=arch_model(returns, vol='ARCH', p=1)
results=model.fit(disp='off')

# summary 

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                      Constant Mean - ARCH Model Results                      
==============================================================================
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                       ARCH   Log-Likelihood:               -1400.95
Distribution:                  Normal   AIC:                           2807.90
Method:            Maximum Likelihood   BIC:                           2821.76
                                        No. Observations:                  752
Date:                Thu, Jul 17 2025   Df Residuals:                      751
Time:                        20:58:27   Df Model:                            1
                                 Mean Model                                
===========================================================================
                 coef    std err          t      P>|t|     95.0% Conf. Int.
---------------------------------------------------------------------------
mu             0.0978  5.518e-02      1.772  7.646e-02 [-1.039e-02,  0.206]
                              Volatility Model                             
===========================================================================
                 coef    std err          t      P>|t|     95.0% Conf. Int.
---------------------------------------------------------------------------
omega          2.1877      0.332      6.584  4.591e-11    [  1.536,  2.839]
alpha[1]       0.1224  7.467e-02      1.639      0.101 [-2.394e-02,  0.269]
===========================================================================

Covariance estimator: robust
"""

analysis 

mu- 0.1260   => avg daily return 

omega =	1.6656  => the long run base level of variance 

alpha[1] = 0.4211  => how much yesterdays squared shock impacts todays variance 

In [7]:
# forecast 5 days ahead 

forecast= results.forecast(horizon=5)
predicted_variance = forecast.variance


# volatility = var sqroot

predicted_volatility= predicted_variance**0.5
predicted_volatility






,h.1,h.2,h.3,h.4,h.5
Date,,,,,
2024-12-31,1.479267,1.567022,1.577429,1.578698,1.578853


In [8]:
# calculate avg of predicted volaitilty 

predicted_volatility=[1.479268,	1.567023,	1.577429,	1.578698,	1.578854]
predicted_avg_volatility= sum(predicted_volatility)/len(predicted_volatility)
predicted_avg_volatility

1.5562544

In [9]:
# to compare forecasted volatility with real 

start_date= pd.to_datetime('2024-12-31')
end_date = pd.to_datetime('2025-01-09')



real_df= yf.download('JPM', start_date, end_date)
real_df['retruns']= real_df['Close'].pct_change()*100
real_df = real_df.dropna()

realised_vol = real_df['retruns'].std()*np.sqrt(5)    # since we are predicting for next five day so sqrt (5)
realised_vol

C:\Users\DELL\AppData\Local\Temp\ipykernel_8028\2524937218.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  real_df= yf.download('JPM', start_date, end_date)
[*********************100%***********************]  1 of 1 completed


1.691952213933136

In [10]:
print("ARCH model pred. volt.", predicted_avg_volatility)
print("acutual volatility", realised_vol)

ARCH model pred. volt. 1.5562544
acutual volatility 1.691952213933136


# GARCH

In [11]:
from arch import arch_model

In [12]:
# create and fit ARCH model

model=arch_model(returns, vol='GARCH', p=1 , q=1)
results=model.fit(disp='off')

# summary 

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -1387.47
Distribution:                  Normal   AIC:                           2782.94
Method:            Maximum Likelihood   BIC:                           2801.43
                                        No. Observations:                  752
Date:                Thu, Jul 17 2025   Df Residuals:                      751
Time:                        20:58:50   Df Model:                            1
                                Mean Model                                
==========================================================================
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
mu             0.1146  5.754e-02      1.991  4.648e-02 [1.787e-03,  0.227]
                               Volatility Model                              
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
omega          0.0220  1.702e-02      1.294      0.196 [-1.134e-02,5.537e-02]
alpha[1]       0.0165  1.112e-02      1.481      0.139 [-5.326e-03,3.827e-02]
beta[1]        0.9737  1.217e-02     79.973      0.000      [  0.950,  0.998]
=============================================================================

Covariance estimator: robust
"""

mu	0.1146; estimated avg daily return 
omega	0.0220 : the long run base level of variance 
alpha[1]	0.0165: how much yesterdays shock affects todays shock
beta[1]	0.9737:  how much yesterdays variance affects todays variance

In [13]:
# forecast 5 days ahead 

forecast= results.forecast(horizon=5)
predicted_variance = forecast.variance


# volatility = var sqroot

predicted_volatility= predicted_variance**0.5
predicted_volatility


,h.1,h.2,h.3,h.4,h.5
Date,,,,,
2024-12-31,1.623483,1.622252,1.621032,1.619824,1.618626


In [14]:
# calculate avg of predicted volaitilty 

predicted_volatility=[	1.623484,	1.622253,	1.621033,	1.619824,	1.618627]
predicted_avg_volatility= sum(predicted_volatility)/len(predicted_volatility)
predicted_avg_volatility

1.6210442

In [15]:
# to compare forecasted volatility with real 

start_date= pd.to_datetime('2024-12-31')
end_date = pd.to_datetime('2025-01-09')



real_df= yf.download('JPM', start_date, end_date)
real_df['retruns']= real_df['Close'].pct_change()*100
real_df = real_df.dropna()

realised_vol = real_df['retruns'].std()*np.sqrt(5)
realised_vol

C:\Users\DELL\AppData\Local\Temp\ipykernel_8028\1245339672.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  real_df= yf.download('JPM', start_date, end_date)
[*********************100%***********************]  1 of 1 completed


1.691952213933136

In [16]:
print("GARCH model pred. volt.", predicted_avg_volatility)
print("acutual volatility", realised_vol)

GARCH model pred. volt. 1.6210442
acutual volatility 1.691952213933136


# EWMA 

In [17]:
# import data from yfinace 

df=yf.download('JPM',start='2024-08-01', end='2025-01-01')
df=df['Close']
df

C:\Users\DELL\AppData\Local\Temp\ipykernel_8028\1238891280.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df=yf.download('JPM',start='2024-08-01', end='2025-01-01')
[*********************100%***********************]  1 of 1 completed


Ticker,JPM
Date,
2024-08-01,203.392670
2024-08-02,194.766373
2024-08-05,190.619476
2024-08-06,195.940018
2024-08-07,195.998703
...,...
2024-12-24,238.440506
2024-12-26,239.257263
2024-12-27,237.318726


In [36]:
df = yf.download('JPM', start='2024-01-01', end='2025-01-01')

df = df[['Close']]



df 


C:\Users\DELL\AppData\Local\Temp\ipykernel_8028\481764788.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download('JPM', start='2024-01-01', end='2025-01-01')
[*********************100%***********************]  1 of 1 completed


Price,Close
Ticker,JPM
Date,
2024-01-02,165.382812
2024-01-03,164.661972
2024-01-04,165.754700
2024-01-05,166.586334
2024-01-08,166.344574
...,...
2024-12-24,238.440521
2024-12-26,239.257263


In [37]:
# calculate daily return

df['returns']=df.pct_change()

df= df.dropna()

# set lemda value for EWMA model 

lamda= 0.94



In [41]:
ewma_var = []
var_t = df['returns'].var()  # Initial variance

for ret in df['returns']:
    var_t = lamda * var_t + (1 - lamda) * (ret ** 2)  # update var_t
    ewma_var.append(var_t)

df['ewma_vol'] = np.sqrt(ewma_var)  # convert variance to volatility
latest_daily_vol = df['ewma_vol'].iloc[-1]

latest_daily_vol


C:\Users\DELL\AppData\Local\Temp\ipykernel_8028\1447613761.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ewma_vol'] = np.sqrt(ewma_var)  # convert variance to volatility


0.014839785511052694

In [39]:
# initialize variance and calculate EWMA 

ewma_var=[]
var_t= df['returns'].var()

for ret in df['returns']:
    variance_t_plus1 = lamda*var_t + (1-lamda)* (ret**2)
    ewma_var.append(variance_t_plus1)




# Volatility = sqrt(variance)

df['ewma_vol'] = np.sqrt(ewma_var)



# predicted volatility 

latest_daily_vol = df['ewma_vol'].iloc[-1]  # predicted volatility
# EWMA can only be used for 1-day prediction
latest_daily_vol




C:\Users\DELL\AppData\Local\Temp\ipykernel_8028\2270905487.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ewma_vol'] = np.sqrt(ewma_var)


0.01436954737061098

In [40]:
# EWMA volatility predicted on 2024-12-31
forecasted_vol = df['ewma_vol'].iloc[-1]  # for 2025-01-02

# Get realized return on 2025-01-02
real_df = yf.download('JPM', start='2025-01-01', end='2025-01-04')
real_df['returns'] = real_df['Close'].pct_change()
real_df = real_df.dropna()

realized_return = real_df['returns'].iloc[0]  # return on 2025-01-02
realized_vol = abs(realized_return)  # one-day realized volatility estimate


realized_vol


C:\Users\DELL\AppData\Local\Temp\ipykernel_8028\779457935.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  real_df = yf.download('JPM', start='2025-01-01', end='2025-01-04')
[*********************100%***********************]  1 of 1 completed

0.01366670745715104